# LSTM

Test of simple MLP models with different amount of neurons, hidden layers and size of input vector

## Load Data

In [174]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [175]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [176]:
df = pd.read_csv("data_N_6000_noise_level_0.05.csv")
df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [177]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        self.X_, self.y_ = self.__make_stack(dataframe)
        print(self.X_.shape)
        print(self.y_.shape)

    def __len__(self):
        return len(self.X_)

    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]

    def __make_stack(self, df: pd.DataFrame):
        seq_amount = df.shape[0] - 2 * self.n_dots - 1
        X = torch.zeros((seq_amount, self.n_dots, self.n_dot_parameters), dtype=torch.float32)
        y = torch.zeros((seq_amount, self.n_dots, self.n_dot_parameters), dtype=torch.float32)
        for i in range(seq_amount):
            X[i, :, :] = torch.reshape(torch.tensor(df.values[i:i+self.n_dots, :], dtype=torch.float32), (1, self.n_dots, self.n_dot_parameters))
            y[i, :, :] = torch.reshape(torch.tensor(df.values[i+self.n_dots:i+2*self.n_dots, :], dtype=torch.float32), (1, self.n_dots, self.n_dot_parameters))

        # stacks = [[df.iloc[:-self.n_dots]]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return (X, y)

In [178]:
N_DOTS = 20
BATCH_SIZE = 8
N_DOT_PARAMETERS = 4
N_LSTM_LAYERS = N_DOTS

In [179]:
dataset = SequentDataset(df_diff[0:5000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[5000:6000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)

torch.Size([4959, 20, 4])
torch.Size([4959, 20, 4])
torch.Size([958, 20, 4])
torch.Size([958, 20, 4])


In [180]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=N_DOTS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=N_DOTS, shuffle=False)
test_loader = DataLoader(test, batch_size=N_DOTS, shuffle=False)

## Create Model

In [181]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=100, output_size=4, num_layers=10):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        x = x.to(device)
        batch_size = x.size(0)
        self.h = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        self.h = self.h.to(device)
        self.c = torch.zeros(self.num_layers, batch_size, self.hidden_layer_size).requires_grad_()
        self.c = self.c.to(device)
        out, _ = self.lstm(x, (self.h, self.c))

        # print(out.size())

        # Pass through fully connected layer
        out = self.linear(out)  # We want the output corresponding to the last time step
        return out


In [182]:
model = LSTMModel(input_size=N_DOT_PARAMETERS, num_layers=N_LSTM_LAYERS).to(device)
loss_model = nn.MSELoss(reduction='mean')
# loss_model = nn.L1Loss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=5, gamma=0.95)

In [183]:
model

LSTMModel(
  (lstm): LSTM(4, 100, num_layers=20, batch_first=True)
  (linear): Linear(in_features=100, out_features=4, bias=True)
)

In [184]:
EPOCH = 250
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    train_loss = []
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)

        y_pred = model(X)
        loss = loss_model(y_pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)

    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    lr_scheduler.step()
    lr = lr_scheduler.get_last_lr()
    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

Epoch [1/250], train_loss = 0.0144, val_loss = 0.0019
Epoch [2/250], train_loss = 0.0019, val_loss = 0.0020
Epoch [3/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [4/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [5/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [6/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [7/250], train_loss = 0.0019, val_loss = 0.0020
Epoch [8/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [9/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [10/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [11/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [12/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [13/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [14/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [15/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [16/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [17/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [18/250], train_loss = 0.0019, val_loss = 0.0019
Epoch [19/250], tra

In [185]:
pred.size()

torch.Size([11, 20, 4])

In [186]:
a = [1, 2, 3, 4]
a[:-1]

[1, 2, 3]

In [187]:
start, _  = dataset[0]
# print(start.size())
# start = torch.reshape(start, (1, start.size(0)))
predictions = []
model.eval()

for X, y in train_loader:
    with torch.no_grad():
        for i in range(N_DOTS, 500):
            y_pred = model(X)
            predictions.append(y_pred[0, -1, :])
            X[0, :-1, :] = X.clone()[0, 1:, :]
            X[0, -1, :] = y_pred[0, -1, :]

    break

In [188]:
start

tensor([[ 1.0337e+00,  6.7231e-02,  8.3389e-02,  7.3115e-02],
        [ 9.6222e-01, -2.0753e-02, -7.6913e-02, -4.3385e-04],
        [ 9.9967e-01,  1.3553e-02, -1.1785e-02, -4.0977e-02],
        [ 1.0167e+00, -1.5033e-02,  5.9923e-03,  3.6317e-02],
        [ 1.0725e+00, -3.3989e-02,  5.9333e-02,  1.5001e-02],
        [ 9.8092e-01,  3.1740e-02, -7.1350e-02, -4.9139e-02],
        [ 9.4035e-01, -1.7860e-03,  8.0157e-03,  3.4894e-02],
        [ 1.0501e+00, -4.8864e-02, -1.8851e-02, -1.9254e-02],
        [ 9.8786e-01,  1.5131e-02,  1.8590e-02, -2.7577e-02],
        [ 1.0439e+00, -9.7870e-04, -2.6854e-02, -1.2646e-02],
        [ 9.8502e-01, -2.3867e-02, -2.5049e-02, -3.2565e-02],
        [ 9.6441e-01,  7.7511e-02,  1.4802e-02,  3.4769e-02],
        [ 1.0205e+00, -8.9182e-02,  3.6208e-02, -4.5491e-02],
        [ 9.4417e-01,  3.7399e-02, -4.7262e-02,  7.3084e-02],
        [ 1.0558e+00, -6.4765e-03, -2.4762e-02, -8.9856e-02],
        [ 1.0338e+00, -3.4726e-02,  3.1416e-02,  6.8764e-02],
        

In [189]:
len(predictions)

480

In [190]:
y.size()

torch.Size([20, 20, 4])

In [191]:
p_x = pd.DataFrame(np.vstack(list(map(lambda x: x.cpu().detach().numpy(), predictions))), columns=["Time", "X", "Y", "Z"])
_, y  = dataset[:500]
p_y = pd.DataFrame(y.cpu().detach().numpy()[:, 0, :], columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [192]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [193]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()